In [174]:
# You can set DOMO_CLIENT_ID and DOMO_CLIENT_SECRET as environment variables
# or enter them interactively. Optionally set DOMO_API_HOST (defaults to api.domo.com).

import os
import getpass
from dotenv import load_dotenv
from pydomo import Domo

# load .env if present
load_dotenv()

CLIENT_ID = os.getenv('DOMO_CLIENT_ID') or input('DOMO_CLIENT_ID: ').strip()
CLIENT_SECRET = os.getenv('DOMO_CLIENT_SECRET') or getpass.getpass('DOMO_CLIENT_SECRET: ')
API_HOST = os.getenv('DOMO_API_HOST', 'api.domo.com').strip()

# Create authenticated client
domo = Domo(CLIENT_ID, CLIENT_SECRET, api_host=API_HOST, logger_name='pydomo')
print('Authenticated to Domo API host:', API_HOST)


Authenticated to Domo API host: api.domo.com


In [45]:
# Helper functions to list pages, fetch page metadata, collections, and cards
import pandas as pd
from typing import Dict, Any, List
from datetime import datetime

def list_pages_df(domo: 'Domo') -> pd.DataFrame:
    """Return DataFrame of pages (dashboards) accessible by the API client."""
    pages = domo.page_list()
    rows = []
    for p in pages:
        rows.append({
            'id': p.get('id'),
            'title': p.get('title') or p.get('name'),
            'description': p.get('description'),
            'parentId': p.get('parentId'),
            'childrenCount': len(p.get('children', [])) if isinstance(p.get('children'), list) else None,
            'isFavorite': p.get('favorite') if 'favorite' in p else None,
            'url': p.get('url') or p.get('link')
        })
    return pd.DataFrame(rows)

def page_summary(domo: 'Domo', page_id: int) -> Dict[str, Any]:
    """Return a brief summary + raw metadata/collections for a page."""
    meta = domo.page_get(page_id)
    collections = domo.page_get_collections(page_id)
    return {
        'page_id': meta.get('id'),
        'title': meta.get('title') or meta.get('name'),
        'description': meta.get('description'),
        'collection_count': len(collections) if isinstance(collections, list) else None,
        'raw_meta': meta,
        'raw_collections': collections,
    }

def collections_df(domo: 'Domo', page_id: int) -> pd.DataFrame:
    """Return DataFrame describing collections on a page."""
    cols = domo.page_get_collections(page_id)
    out = []
    for c in cols or []:
        cards = c.get('cards', []) if isinstance(c, dict) else []
        out.append({
            'collection_id': c.get('id'),
            'collection_title': c.get('title') or c.get('name'),
            'card_count': len(cards) if isinstance(cards, list) else None
        })
    return pd.DataFrame(out)

def cards_df_from_collections(domo: 'Domo', page_id: int) -> pd.DataFrame:
    """Build DataFrame of cards found inside the page's collections."""
    cols = domo.page_get_collections(page_id)
    rows = []
    for c in cols or []:
        col_id = c.get('id')
        col_title = c.get('title') or c.get('name')
        for card in (c.get('cards') or []):
            if isinstance(card, dict):
                rows.append({
                    'collection_id': col_id,
                    'collection_title': col_title,
                    'card_id': card.get('id') or card.get('cardId'),
                    'card_title': card.get('title') or card.get('name'),
                    'card_type': card.get('type'),
                })
            else:
                # card may be an ID only
                rows.append({
                    'collection_id': col_id,
                    'collection_title': col_title,
                    'card_id': card,
                    'card_title': None,
                    'card_type': None,
                })
    return pd.DataFrame(rows)

def export_results(pages: pd.DataFrame, collections: pd.DataFrame, cards: pd.DataFrame, base_path: str = './') -> dict:
    """Export CSVs and a multi-sheet Excel file. Returns paths."""
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    out_csv_pages = f"{base_path}domo_pages_{ts}.csv"
    out_csv_collections = f"{base_path}domo_page_collections_{ts}.csv"
    out_csv_cards = f"{base_path}domo_page_cards_{ts}.csv"
    pages.to_csv(out_csv_pages, index=False)
    collections.to_csv(out_csv_collections, index=False)
    cards.to_csv(out_csv_cards, index=False)

    out_xlsx = f"{base_path}domo_dashboard_details_{ts}.xlsx"
    with pd.ExcelWriter(out_xlsx) as xl:
        pages.to_excel(xl, sheet_name='Pages', index=False)
        collections.to_excel(xl, sheet_name='Collections', index=False)
        cards.to_excel(xl, sheet_name='Cards', index=False)
    return {
        'pages_csv': out_csv_pages,
        'collections_csv': out_csv_collections,
        'cards_csv': out_csv_cards,
        'xlsx': out_xlsx,
    }


In [175]:
# List all pages you have access to and display first rows
pages_df = list_pages_df(domo)
print('Total pages found:', len(pages_df))
pages_df.head(30)  # use display(pages_df) in Jupyter if you prefer


Total pages found: 4


,id,title,description,parentId,childrenCount,isFavorite,url
0,333365150,Example Sales Dashboard,None,None,0,None,None
1,51087845,Example Marketing Dashboard,None,None,0,None,None
2,1439666643,Ecommerce Dashboard,None,None,0,None,None
3,1195479659,Sales Dashboard,None,None,0,None,None


In [ ]:
from pydomo import Domo
from dotenv import load_dotenv
import os
from importlib import reload
import pandas as pd

reload(dotenv)
load_dotenv(override=True)

CLIENT_ID = os.getenv('DOMO_CLIENT_ID') or input('DOMO_CLIENT_ID: ').strip()
CLIENT_SECRET = os.getenv('DOMO_CLIENT_SECRET') or getpass.getpass('DOMO_CLIENT_SECRET: ')
API_HOST = os.getenv('DOMO_API_HOST', 'api.domo.com').strip()

# Create authenticated client
domo = Domo(CLIENT_ID, CLIENT_SECRET, api_host=API_HOST, logger_name='pydomo')

def list_pages_df(domo: 'Domo') -> pd.DataFrame:
    """Return DataFrame of pages (dashboards) accessible by the API client."""
    pages = domo.page_list()
    rows = []
    for p in pages:
        rows.append({
            'id': p.get('id'),
            'title': p.get('title') or p.get('name'),
            'description': p.get('description'),
            'parentId': p.get('parentId'),
            'childrenCount': len(p.get('children', [])) if isinstance(p.get('children'), list) else None,
            'isFavorite': p.get('favorite') if 'favorite' in p else None,
            'url': p.get('url') or p.get('link')
        })
    return pd.DataFrame(rows)


pages_df = list_pages_df(domo)
print('Total pages found:', len(pages_df))
pages_df.head(30)
print(pages_df)

#Chane dashboard id to get the data
page_id = pages_df.loc[
    pages_df['title'].str.lower().str.strip() == 'Ecommerce Dashboard',
    'id'
].squeeze()

if pd.notna(page_id):
    print(f"Page ID: {page_id}")
else:
    print("Page with title 'Ecommerce Dashboard' not found.")
    

Total pages found: 4
           id                        title description parentId  \
0   333365150      Example Sales Dashboard        None     None   
1    51087845  Example Marketing Dashboard        None     None   
2  1439666643          Ecommerce Dashboard        None     None   
3  1195479659              Sales Dashboard        None     None   

   childrenCount isFavorite   url  
0              0       None  None  
1              0       None  None  
2              0       None  None  
3              0       None  None  
Page ID: 1439666643


In [1]:
#DOMO API code starts here
import requests
from dotenv import load_dotenv
import os
from importlib import reload
import dotenv

reload(dotenv)
load_dotenv(override=True)
instance = os.getenv("DOMO_INSTANCE")
developer_token = os.getenv("DOMO_DEVELOPER_TOKEN")
page_id = 1195479659
print(f"Page ID: {page_id}")

url = f"https://{instance}.domo.com/api/content/v1/pages"
headers = {
    "X-DOMO-Developer-Token": developer_token,
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)
if response.status_code == 200:
    page_details = response.json()
    print("Page Details:")
    print(page_details)
else:
    print("Failed to fetch page details:", response.status_code, response.text)


/Users/aswinkanna/Domo test/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Page ID: 1195479659
Page Details:
[{'id': '190576085', 'title': 'Sales Dashboard Advanced', 'type': 'page', 'locked': False, 'sharedView': True, 'pageVisible': True, 'owner': 1598896440, 'owners': [{'id': 1598896440, 'type': 'USER', 'displayName': 'Harini Ganga'}], 'isOwner': False, 'canAddCard': False, 'bibHiddenPage': False, 'hasAccess': True}, {'id': '333365150', 'title': 'Example Sales Dashboard', 'type': 'page', 'locked': False, 'sharedView': True, 'pageVisible': True, 'owner': 27, 'owners': [{'id': 27, 'type': 'USER', 'displayName': 'DomoSupport'}], 'isOwner': False, 'canAddCard': False, 'bibHiddenPage': False, 'hasAccess': True}, {'id': '51087845', 'title': 'Example Marketing Dashboard', 'type': 'page', 'locked': False, 'sharedView': True, 'pageVisible': True, 'owner': 27, 'owners': [{'id': 27, 'type': 'USER', 'displayName': 'DomoSupport'}], 'isOwner': False, 'canAddCard': False, 'bibHiddenPage': False, 'hasAccess': True}, {'id': '1439666643', 'title': 'Ecommerce Dashboard', 'ty

In [123]:
#DOMO API code starts here
import requests
from dotenv import load_dotenv
import os
from importlib import reload
import dotenv

reload(dotenv)
load_dotenv(override=True)
instance = os.getenv("DOMO_INSTANCE")
developer_token = os.getenv("DOMO_DEVELOPER_TOKEN")
page_id = 1195479659
print(f"Page ID: {page_id}")

url = f"https://{instance}.domo.com/api/content/v1/pages/{page_id}"
headers = {
    "X-DOMO-Developer-Token": developer_token,
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)
if response.status_code == 200:
    page_details = response.json()
    print("Page Details:")
    print(page_details)
else:
    print("Failed to fetch page details:", response.status_code, response.text)


Page ID: 1195479659
Page Details:
{'pageId': 1195479659, 'ownerId': 1598896440, 'owners': [{'id': 1598896440, 'type': 'USER', 'displayName': 'Harini Ganga'}], 'type': 'page', 'title': 'Sales Dashboard', 'pageName': 'Sales Dashboard1', 'created': 1757308949000, 'updated': 1757412031577, 'locked': False, 'mobileEnabled': True, 'sharedViewPage': True, 'virtualPage': False, 'isOwner': False}


In [138]:
url = f"https://{instance}.domo.com/api/content/v1/pages/{page_id}/cards"

response = requests.get(url, headers=headers)
print("resp",response)
if response.status_code == 200:
    cards = response.json()
    print(f"Found {len(cards)} cards in 'Options Report'")
    
    for card in cards:
        print(f"- Card ID: {card['id']}, Title: {card['title']}")
else:
    print("Failed to fetch cards:", response.status_code, response.text)


resp <Response [200]>
Found 9 cards in 'Options Report'
- Card ID: 1768126599, Title: Harini Ganga's Notebook Sep 8, 2025
- Card ID: 1646345374, Title: sales3
- Card ID: 1429903375, Title: Sales By Category
- Card ID: 2102932184, Title: Sales by area
- Card ID: 1150560925, Title: Sales by City
- Card ID: 1628734641, Title: Sales vs Profit
- Card ID: 405522535, Title: Regional Sales Report
- Card ID: 1370501068, Title: Sales Heatmap
- Card ID: 270078103, Title: Product Performance Matrix


In [125]:
url = f"https://{instance}.domo.com/api/content/v1/pages/{page_id}/cards"
response = requests.get(url, headers=headers)
print(response.json())

[{'urn': '1768126599', 'id': 1768126599, 'type': 'Text', 'created': 1757323423, 'badgeUpdated': 1757323958000, 'creatorId': 1598896440, 'ownerId': 1598896440, 'title': "Harini Ganga's Notebook Sep 8, 2025", 'active': True, 'allowTableDrill': True, 'locked': False}, {'urn': '1646345374', 'id': 1646345374, 'type': 'kpi', 'created': 1757321784, 'badgeUpdated': 1757397291000, 'creatorId': 1598896440, 'ownerId': 1598896440, 'title': 'sales3', 'active': True, 'allowTableDrill': True, 'locked': False}, {'urn': '1429903375', 'id': 1429903375, 'type': 'kpi', 'created': 1757392236, 'badgeUpdated': 1757397305000, 'creatorId': 1598896440, 'ownerId': 1598896440, 'title': 'Sales By Category', 'active': True, 'allowTableDrill': True, 'locked': False}, {'urn': '2102932184', 'id': 2102932184, 'type': 'kpi', 'created': 1757321181, 'badgeUpdated': 1757397616000, 'creatorId': 1598896440, 'ownerId': 1598896440, 'title': 'Sales by area', 'active': True, 'allowTableDrill': True, 'locked': False}, {'urn': '11

In [ ]:
for card in cards:
    card_id = card['id']
    url = f"https://{instance}.domo.com/api/content/v1/cards/{card_id}/data"
    response = requests.get(url, headers=headers)
    
    print(card_id,response.status_code, response.text)
    


405522535 200 {"data":{"datasource":"cd8b7651-9d72-42c2-bfe2-456113df8d1c","columns":["Region","Profit Margin","Segment"],"aliases":["Region","Profit Margin","Segment"],"formats":[null,{"type":"number","format":"#,###.00","commas":true,"precision":2},null],"mappings":["ITEM","VALUE","SERIES"],"metadata":[{"type":"STRING","dataSourceId":"cd8b7651-9d72-42c2-bfe2-456113df8d1c","maxLength":-1,"minLength":-1,"periodIndex":0,"aggregated":false,"label":"Region","column":"Region"},{"type":"DOUBLE","dataSourceId":"cd8b7651-9d72-42c2-bfe2-456113df8d1c","maxLength":-1,"minLength":-1,"periodIndex":0,"aggregated":true,"label":"Profit Margin","column":"Profit Margin"},{"type":"STRING","dataSourceId":"cd8b7651-9d72-42c2-bfe2-456113df8d1c","maxLength":-1,"minLength":-1,"periodIndex":0,"aggregated":false,"label":"Segment","column":"Segment"}],"fromcache":true,"rows":[["East",11.449844347422735,"Consumer"],["South",15.069346533197582,"Consumer"],["Central",-1.388085888121337,"Consumer"],["West",16.19406

In [167]:
cards = [{"id": 405522535}] 
for card in cards:
    card_id = card['id']
    url = f"https://{instance}.domo.com/api/content/v1/cards/{card_id}/data"
    response = requests.get(url, headers=headers)
    print(card_id, response.status_code)

    if response.status_code == 200:
        data = response.json().get("data", {})
        metadata = data.get("metadata", [])

        dimensions = [m["column"] for m in metadata if not m.get("aggregated", False)]
        measures   = [m["column"] for m in metadata if m.get("aggregated", False)]

        print(f"Card {card_id} → Dimensions: {dimensions}, Measures: {measures}")
    else:
        print("Error:", response.text)

405522535 200
Card 405522535 → Dimensions: ['Region', 'Segment'], Measures: ['Profit Margin']


In [169]:
import requests
import os
from dotenv import load_dotenv

# Load environment variables (DOMO_INSTANCE, DOMO_DEVELOPER_TOKEN)
load_dotenv()
instance = os.getenv("DOMO_INSTANCE")
developer_token = os.getenv("DOMO_DEVELOPER_TOKEN")

# Replace with your dataset ID (sales3 dataset ID from Domo)
dataset_id = "cd8b7651-9d72-42c2-bfe2-456113df8d1c"

url = f"https://{instance}.domo.com/api/query/v1/calculatedFields/{dataset_id}"
headers = {
    "X-DOMO-Developer-Token": developer_token,
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    calculated_fields = response.json()
    for field in calculated_fields:
        name = field.get("name")
        formula = field.get("expression")
        print(f"Calculated Field: {name} → Formula: {formula}")
else:
    print("Error:", response.status_code, response.text)


Error: 404 {"status":404,"statusReason":"Not Found","message":"No static resource api/query/v1/calculatedFields/cd8b7651-9d72-42c2-bfe2-456113df8d1c.","toe":"K329TT3M2Y-FMJN8-LAVN1"}


In [ ]:
user_id = "1598896440"
url = f"https://{instance}.domo.com/api/identity/v1/users/{user_id}"
response = requests.get(url, headers=headers)
print(response.json())
if response.status_code == 200:
    response_json = response.json()
    user = response_json['users'][0]

    user_id = user['id']
    name = user['displayName']
    username = user['userName']
    email = user['emailAddress']
    role_id = user['roleId']

    print(f"ID: {user_id}")
    print(f"Name: {name}")
    print(f"Username: {username}")
    print(f"Email: {email}")
    print(f"Role ID: {role_id}")
else:
    print(f"Error {response.status_code}: {response.text}")
#DATASET access
# --- DATASET ACCESS ---
url = f"https://{instance}.domo.com/api/users/v1/{user_id}/datasets"
response = requests.get(url, headers=headers)
if response.status_code == 200 and response.text:
    datasets = response.json()
    for ds in datasets:
        print(f"Dataset: {ds.get('name')}, ID: {ds.get('id')}")
else:
    print(f"No datasets or error: {response.status_code}")

# --- CARD ACCESS ---
url = f"https://{instance}.domo.com/api/users/v1/{user_id}/cards"
response = requests.get(url, headers=headers)
if response.status_code == 200 and response.text:
    cards = response.json()
    for card in cards:
        print(f"Card: {card.get('name')}, ID: {card.get('id')}")
else:
    print(f"No cards or error: {response.status_code}")

# --- GROUPS ---
url = f"https://{instance}.domo.com/api/users/v1/{user_id}/groups"
response = requests.get(url, headers=headers)
if response.status_code == 200 and response.text:
    groups = response.json()
    for group in groups:
        print(f"Group: {group.get('name')}, ID: {group.get('id')}")
else:
    print(f"No groups or error: {response.status_code}")


{'users': [{'attributes': [{'key': 'id', 'values': [1598896440]}, {'key': 'displayName', 'values': ['Harini Ganga']}, {'key': 'userName', 'values': ['harini@squareshift.co']}, {'key': 'emailAddress', 'values': ['harini@squareshift.co']}, {'key': 'roleId', 'values': [1]}], 'id': 1598896440, 'displayName': 'Harini Ganga', 'userName': 'harini@squareshift.co', 'roleId': 1, 'emailAddress': 'harini@squareshift.co'}]}
ID: 1598896440
Name: Harini Ganga
Username: harini@squareshift.co
Email: harini@squareshift.co
Role ID: 1
No groups or error: 404


In [ ]:
import requests


instance = os.getenv("DOMO_INSTANCE")
developer_token = os.getenv("DOMO_DEVELOPER_TOKEN")
# Dataset ID
dataset_id = "cd8b7651-9d72-42c2-bfe2-456113df8d1c"

# API endpoint
# url = f"https://{instance}.domo.com/api/data/v3/datasources/{dataset_id}?part=core,permission"
url = f"https://{instance}.domo.com/api/data/v3/datasources/"#ALL DATASETS API

# Headers
headers = {
    "X-DOMO-Developer-Token": developer_token,
    "Content-Type": "application/json"
}

# GET request
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    print("✅ Dataset details and permissions retrieved successfully")
    print(data)  # full JSON
else:
    print(f"❌ Error {response.status_code}: {response.text}")


✅ Dataset details and permissions retrieved successfully
{'dataSources': [{'id': '7f836889-b681-4a75-94da-e29be19ab3fa', 'displayType': 'domo-federateddata', 'dataProviderType': 'domo-federateddata', 'type': 'domo-federateddata', 'name': 'Sample Marketing Data', 'description': 'Sample dataset with common marketing KPIs', 'owner': {'id': '1598896440', 'name': 'Harini Ganga', 'type': 'USER', 'group': False}, 'status': 'SUCCESS', 'created': 1756872252000, 'lastTouched': 1757540260000, 'lastUpdated': 1757540259731, 'rowCount': 1000, 'columnCount': 16, 'cardInfo': {'cardCount': 0, 'cardViewCount': 0}, 'properties': {'formulas': {'formulas': {}}}, 'state': 'IDLE', 'validConfiguration': True, 'validAccount': True, 'streamId': 1, 'transportType': 'PUBLISH', 'adc': False, 'adcExternal': False, 'masked': False, 'currentUserFullAccess': True, 'permissions': 'READ', 'hidden': False, 'scheduleActive': True}, {'id': 'e0edd4e8-3b38-4eb6-8284-fe4081f1e957', 'displayType': 'domo-federateddata', 'dataPr

In [170]:

url = f"https://{instance}.domo.com/api/query/v1/functions/statistics"
headers = {
    "X-DOMO-Developer-Token": developer_token,
    "Content-Type": "application/json"
}

res = requests.get(url, headers=headers)
print(res.status_code, res.json())

cards = [{"id": 405522535}, {"id": 1370501068}]  # replace with your card IDs

for card in cards:
    card_id = card['id']
    url = f"https://{instance}.domo.com/api/content/v1/cards/{card_id}/data"
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        card_info = response.json()
        card_name = card_info.get("name", "Unknown Card")
        beast_modes = card_info.get("beastModes", [])
        
        print(f"\nCard: {card_name} (ID: {card_id})")
        if beast_modes:
            for bm in beast_modes:
                print(f"  - Beast Mode Name: {bm.get('name')}")
                print(f"    Formula: {bm.get('formula')}")
                print(f"    Agg Type: {bm.get('aggregation', 'N/A')}")
        else:
            print("  No Beast Modes found.")
    else:
        print(f"Error fetching card {card_id}: {response.status_code}, {response.text}")


200 {'total': 87, 'onDatasets': 45, 'onCards': 42, 'inVisualizations': 51, 'locked': 0, 'invalid': 0, 'invalidLink': 0, 'archived': 0}

Card: Unknown Card (ID: 405522535)
  No Beast Modes found.

Card: Unknown Card (ID: 1370501068)
  No Beast Modes found.


In [119]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()
instance = os.getenv("DOMO_INSTANCE")
access_token = os.getenv("DOMO_DEVELOPER_TOKEN")  # from the step where you requested with scope: "data user dashboard alert"
print(access_token,instance)
card_id = 587394437

url = f"https://{instance}.domo.com/api/social/v4/alerts"
headers = {
    "Authorization": f"bearer {access_token}",
    "Content-Type": "application/json"
}

resp = requests.get(url, headers=headers)
print("Status:", resp.status_code)
alerts = resp.json() if resp.status_code == 200 else []
print(alerts)


DDCI056358f88866b6c7081a5d2f0829351f790abf7c313d3a86 squareshift-co-34
Status: 401
[]


In [61]:
import os
import requests

instance = os.getenv("DOMO_INSTANCE")  # e.g., "squareshift-co-35"
developer_token = os.getenv("DOMO_DEVELOPER_TOKEN")
dataflow_id = "1"  # From your URL

url = f"https://{instance}.domo.com/api/dataprocessing/v1/dataflows/{dataflow_id}"
headers = {
    "X-DOMO-Developer-Token": developer_token,
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    magic_etl = response.json()
    print("Magic ETL Name:", magic_etl.get("name"))
    print("Magic ETL Type:", magic_etl.get("type"))
    print("Datasets:", magic_etl.get("datasets"))
    print("Definition:", magic_etl)  # This will include steps if available
else:
    print("Failed to fetch Magic ETL:", response.status_code, response.text)


Magic ETL Name: New ETL Transform
Magic ETL Type: None
Datasets: None
Definition: {'gui': {'version': '1.0', 'canvases': {'default': {'canvasSettings': {'coarserGrid': True, 'backgroundVariant': 'None', 'hideCoarserGridPopUp': False}, 'elements': [{'x': 224, 'y': 128, 'id': 'LoadFromVault-5b5c6ebc-6a50-4876-9ef2-c53a9ef45a75', 'type': 'Tile', 'color': 3701798}, {'x': 352, 'y': 128, 'id': 'Filter-8f94ee0d-51d0-4030-b696-6a48b633ceb3', 'type': 'Tile'}, {'x': 480, 'y': 128, 'id': 'PublishToVault-613b8a4e-6a74-45cf-bbba-364f54e441a6', 'type': 'Tile'}]}}, 'useGraphUI': True}, 'id': 1, 'name': 'New ETL Transform', 'dapDataFlowId': 'f5e7e8d4-39f3-4125-bc57-ac2060a288b0', 'responsibleUserId': 1471496075, 'runState': 'ENABLED', 'lastExecution': {'id': 1, 'onboardFlowId': 1, 'previewRows': 0, 'dapDataFlowExecutionId': 'bbd0f488-b7fd-47a5-b685-674c11977589', 'beginTime': 1757310385000, 'endTime': 1757310414000, 'lastUpdated': 1757310414000, 'failed': True, 'state': 'REJECTED', 'errors': [{'code':

In [56]:
card_id = 2142853841
export_url = f"https://{instance}.domo.com/api/content/v1/cards/{card_id}/export?exportType=PNG"

export_response = requests.get(export_url, headers=headers)

if export_response.status_code == 200:
    with open("card_export.png", "wb") as f:
        f.write(export_response.content)
    print("Card exported as card_export.png")
else:
    print("Failed to export card:", export_response.status_code, export_response.text)


Failed to export card: 405 {"status":405,"statusReason":"Method Not Allowed","message":"Method Not Allowed","toe":"0R3UF3A5D7-HSWVG-MQMJL"}


In [53]:
import os
import requests
from dotenv import load_dotenv

# Load env variables
load_dotenv()

instance = os.getenv("DOMO_INSTANCE")
developer_token = os.getenv("DOMO_DEVELOPER_TOKEN")
card_id = 2142853841

url = f"https://{instance}.domo.com/api/content/v1/cards/{card_id}"
headers = {
    "X-DOMO-Developer-Token": developer_token,
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    card = response.json()
    print("Card Title:", card.get("title"))
    print("Card Type:", card.get("type"))  # e.g., 'chart', 'kpi', 'table', 'file'
    print("Dataset ID:", card.get("dataSourceId"))
else:
    print("Failed to fetch card details:", response.status_code, response.text)


Failed to fetch card details: 405 {"status":405,"statusReason":"Method Not Allowed","message":"Method Not Allowed","toe":"AABFM90Y1H-N5BKH-KQC57"}


In [39]:
# page = domo.pages.get(page['id'])

#         # List pages
# page_list = list(domo.pages.list())
collection_list = domo.pages.get_collections("137519751")

print(collection_list)

[]


In [35]:
import requests
from requests.auth import HTTPBasicAuth
from pydomo import Domo

# Step 1: Get OAuth Token using your client credentials
client_id = CLIENT_ID
client_secret = CLIENT_SECRET

auth_url = "https://api.domo.com/oauth/token"
auth_data = {"grant_type": "client_credentials", "scope": "data"}
auth_response = requests.post(auth_url, auth=HTTPBasicAuth(client_id, client_secret), data=auth_data)

if auth_response.status_code != 200:
    raise Exception(f"Failed to get token: {auth_response.text}")

access_token = auth_response.json()["access_token"]

# Step 2: Initialize Domo client (for pages API)
domo = Domo(client_id=client_id, client_secret=client_secret, api_host="api.domo.com")

# Step 3: Get the page details to extract card IDs
page_id = 137519751  # Replace with your dashboard ID
page_details = domo.pages.get(page_id)
card_ids = page_details.get("cardIds", [])

# Step 4: Find the card by title
target_title = "options report_2025_08_25t17_28_00.xlsx"
headers = {"Authorization": f"Bearer {access_token}"}

found_card = None
for cid in card_ids:
    card_resp = requests.get(f"https://api.domo.com/v1/cards/{cid}", headers=headers)
    if card_resp.status_code == 200:
        card = card_resp.json()
        print("yes",card)
        if card.get("title") == target_title:
            found_card = card
            break

if found_card:
    print("Card found:")
    print(found_card)
else:
    print("Card not found.")


Card not found.


In [7]:
from pydomo import Domo
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Get credentials
CLIENT_ID = os.getenv("DOMO_CLIENT_ID")
CLIENT_SECRET = os.getenv("DOMO_CLIENT_SECRET")
API_HOST = os.getenv("DOMO_API_HOST", "api.domo.com")

# Authenticate
domo = Domo(CLIENT_ID, CLIENT_SECRET, api_host=API_HOST)
print("Connected to Domo!")

Connected to Domo!


In [10]:
pages = domo.pages.list()
for p in pages:
    print(p.keys())

dict_keys(['id', 'name', 'children'])
dict_keys(['id', 'name', 'children'])


In [11]:
pages = domo.pages.list()

# Convert to DataFrame safely
import pandas as pd
pages_df = pd.DataFrame(pages)

# Show available columns
print("Available columns:", pages_df.columns)

# Try using 'title' or 'name' depending on availability
if 'title' in pages_df.columns:
    display_columns = ['id', 'title']
elif 'name' in pages_df.columns:
    display_columns = ['id', 'name']
else:
    display_columns = ['id']  # fallback

print(pages_df[display_columns])


Available columns: Index(['id', 'name', 'children'], dtype='object')
           id                         name
0   137519751      Example Sales Dashboard
1  1851060382  Example Marketing Dashboard


In [40]:
import pandas as pd

# List all dashboards (pages)
pages = domo.pages.list()
pages_df = pd.DataFrame(pages)

print("Available columns:", pages_df.columns)  # Inspect keys

# Adjust based on what key exists (title or name)
name_key = 'title' if 'title' in pages_df.columns else 'name'
dashboard_name = "Example Sales Dashboard"

dashboard_row = pages_df[pages_df[name_key] == dashboard_name]

if not dashboard_row.empty:
    dashboard_id = dashboard_row['id'].iloc[0]
    print("Dashboard found! ID:", dashboard_id)
else:
    raise ValueError("Dashboard not found!")


Available columns: Index(['id', 'name', 'children'], dtype='object')
Dashboard found! ID: 137519751


In [41]:
import pandas as pd

# List all dashboards (pages)
pages = domo.pages.list()
pages_df = pd.DataFrame(pages)

# Find your dashboard
dashboard_name = "Example Sales Dashboard"
dashboard_row = pages_df[pages_df['name'] == dashboard_name]

if not dashboard_row.empty:
    dashboard_id = dashboard_row['id'].iloc[0]
    print(f"Dashboard found! ID: {dashboard_id}")

    # Extract subpages from 'children'
    children = dashboard_row['children'].iloc[0]

    if children:
        subpages_df = pd.DataFrame(children)
        print("\nSubpages Details:")
        print(subpages_df[['id', 'name']])  # Add more columns if available
    else:
        print("\nNo subpages found for this dashboard.")
else:
    raise ValueError("Dashboard not found!")


Dashboard found! ID: 137519751

No subpages found for this dashboard.


In [15]:
dashboard_id


np.int64(137519751)

In [14]:
# Fetch collections (cards are grouped under collections)
collections = domo.pages.get_collections(dashboard_id)

cards = []
for col in collections:
    for card in col.get("cards", []):
        cards.append({
            "collection_id": col.get("id"),
            "collection_title": col.get("title") or col.get("name"),
            "card_id": card.get("id") or card.get("cardId"),
            "card_title": card.get("title") or card.get("name"),
            "card_type": card.get("type")
        })

cards_df = pd.DataFrame(cards)
print(cards_df)


Empty DataFrame
Columns: []
Index: []


In [19]:
page_details = domo.pages.get(dashboard_id)
print(page_details)


{'id': 137519751, 'ownerId': 27, 'name': 'Example Sales Dashboard', 'locked': False, 'collectionIds': [], 'cardIds': [1154368080, 1863447823, 1763326510, 674857235, 249832152, 1886551369, 17410891, 2033830768, 917537435, 488937495, 1613900368, 1523978392, 729138372, 114175563, 1370674614, 1713910732, 982331143, 1985430640, 59753986], 'visibility': {'userIds': [27], 'groupIds': [1324037627]}, 'owners': [{'id': 27, 'type': 'USER', 'displayName': 'DomoSupport'}]}


In [20]:
dashboard_id = 137519751
page_details = domo.pages.get(dashboard_id)

card_ids = page_details.get("cardIds", [])
print("Total cards:", len(card_ids))
print(card_ids)


Total cards: 19
[1154368080, 1863447823, 1763326510, 674857235, 249832152, 1886551369, 17410891, 2033830768, 917537435, 488937495, 1613900368, 1523978392, 729138372, 114175563, 1370674614, 1713910732, 982331143, 1985430640, 59753986]


In [22]:
from tqdm import tqdm  # optional: to show progress
cards = []

for cid in tqdm(card_ids):
    try:
        card = domo.cards.get(cid)
        cards.append({
            "card_id": card.get("id"),
            "card_title": card.get("title") or card.get("name"),
            "card_type": card.get("type"),
            "data_source_id": card.get("dataSourceId") or card.get("datasetId")
        })
    except Exception as e:
        print(f"Failed to fetch card {cid}: {e}")

import pandas as pd
cards_df = pd.DataFrame(cards)
print(cards_df[['card_id', 'card_title', 'card_type']])


ModuleNotFoundError: No module named 'tqdm'